In [1]:
import torch.nn as nn
import torch.nn.functional as F
import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

from pygcn.utils import load_data, accuracy
from pygcn.models import GCN

In [2]:
# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_data(path = "data/cora/")

Loading cora dataset...
Data loaded.


In [4]:
features.shape

torch.Size([2708, 1433])

In [17]:
hidden = 16 
lr = 0.01
epochs = 200
dropout = 0.5
weight_decay = 5e-4
seed = 42

print('nfeatures:', features.shape[1])
print('nHidden:', hidden)
print('nClass:', labels.max().item() + 1)
model = GCN(nfeat=features.shape[1],
            nhid=hidden,
            nclass=labels.max().item() + 1,
            dropout=dropout)

optimizer = optim.Adam(model.parameters(),
                       lr=lr, weight_decay=weight_decay)

nfeatures: 1433
nHidden: 16
nClass: 7


In [11]:
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    print('Using GPU.')
    torch.cuda.manual_seed(seed)
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

Using GPU.


In [12]:
def train(epoch, fastmode=False):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

In [13]:
def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [15]:
# Train model
t_total = time.time()
for epoch in range(epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

Epoch: 0001 loss_train: 1.9113 acc_train: 0.2643 loss_val: 1.8903 acc_val: 0.3867 time: 0.3067s
Epoch: 0002 loss_train: 1.8874 acc_train: 0.2786 loss_val: 1.8776 acc_val: 0.4133 time: 0.0109s
Epoch: 0003 loss_train: 1.8759 acc_train: 0.3429 loss_val: 1.8658 acc_val: 0.4300 time: 0.0088s
Epoch: 0004 loss_train: 1.8640 acc_train: 0.3214 loss_val: 1.8547 acc_val: 0.3767 time: 0.0084s
Epoch: 0005 loss_train: 1.8542 acc_train: 0.3143 loss_val: 1.8439 acc_val: 0.3667 time: 0.0081s
Epoch: 0006 loss_train: 1.8326 acc_train: 0.3643 loss_val: 1.8333 acc_val: 0.3533 time: 0.0089s
Epoch: 0007 loss_train: 1.8318 acc_train: 0.2786 loss_val: 1.8229 acc_val: 0.3500 time: 0.0086s
Epoch: 0008 loss_train: 1.8299 acc_train: 0.3286 loss_val: 1.8128 acc_val: 0.3500 time: 0.0084s
Epoch: 0009 loss_train: 1.8185 acc_train: 0.3000 loss_val: 1.8031 acc_val: 0.3500 time: 0.0089s
Epoch: 0010 loss_train: 1.7992 acc_train: 0.3000 loss_val: 1.7937 acc_val: 0.3500 time: 0.0093s
Epoch: 0011 loss_train: 1.7716 acc_train

Epoch: 0093 loss_train: 0.7738 acc_train: 0.8500 loss_val: 0.9846 acc_val: 0.7767 time: 0.0089s
Epoch: 0094 loss_train: 0.7993 acc_train: 0.8643 loss_val: 0.9802 acc_val: 0.7800 time: 0.0089s
Epoch: 0095 loss_train: 0.7463 acc_train: 0.8429 loss_val: 0.9755 acc_val: 0.7767 time: 0.0090s
Epoch: 0096 loss_train: 0.7820 acc_train: 0.8786 loss_val: 0.9700 acc_val: 0.7767 time: 0.0076s
Epoch: 0097 loss_train: 0.7639 acc_train: 0.8214 loss_val: 0.9631 acc_val: 0.7800 time: 0.0088s
Epoch: 0098 loss_train: 0.7594 acc_train: 0.8500 loss_val: 0.9560 acc_val: 0.7833 time: 0.0088s
Epoch: 0099 loss_train: 0.7147 acc_train: 0.8643 loss_val: 0.9490 acc_val: 0.7867 time: 0.0086s
Epoch: 0100 loss_train: 0.7394 acc_train: 0.8214 loss_val: 0.9435 acc_val: 0.7867 time: 0.0085s
Epoch: 0101 loss_train: 0.7543 acc_train: 0.8714 loss_val: 0.9377 acc_val: 0.7867 time: 0.0085s
Epoch: 0102 loss_train: 0.7172 acc_train: 0.8571 loss_val: 0.9319 acc_val: 0.7867 time: 0.0081s
Epoch: 0103 loss_train: 0.7278 acc_train

Epoch: 0185 loss_train: 0.4353 acc_train: 0.9286 loss_val: 0.7202 acc_val: 0.8067 time: 0.0091s
Epoch: 0186 loss_train: 0.4033 acc_train: 0.9429 loss_val: 0.7229 acc_val: 0.8067 time: 0.0085s
Epoch: 0187 loss_train: 0.3994 acc_train: 0.9214 loss_val: 0.7252 acc_val: 0.8067 time: 0.0089s
Epoch: 0188 loss_train: 0.4137 acc_train: 0.9429 loss_val: 0.7235 acc_val: 0.8067 time: 0.0087s
Epoch: 0189 loss_train: 0.4144 acc_train: 0.9214 loss_val: 0.7187 acc_val: 0.8067 time: 0.0086s
Epoch: 0190 loss_train: 0.3750 acc_train: 0.9357 loss_val: 0.7133 acc_val: 0.8067 time: 0.0082s
Epoch: 0191 loss_train: 0.3957 acc_train: 0.9357 loss_val: 0.7098 acc_val: 0.8067 time: 0.0090s
Epoch: 0192 loss_train: 0.4196 acc_train: 0.9429 loss_val: 0.7082 acc_val: 0.8067 time: 0.0088s
Epoch: 0193 loss_train: 0.4143 acc_train: 0.9286 loss_val: 0.7072 acc_val: 0.8067 time: 0.0084s
Epoch: 0194 loss_train: 0.4256 acc_train: 0.9214 loss_val: 0.7062 acc_val: 0.8067 time: 0.0090s
Epoch: 0195 loss_train: 0.3926 acc_train

In [16]:
# Testing
test()

Test set results: loss= 0.7418 accuracy= 0.8200
